# Compare NDCI with waterbody area across time
**What does this notebook do?** 

This notebooks produces plots which compare the inundated area of a waterbody across time with mean values of the Normalised Difference Chlorophyll Index (NDCI) index.

**Requirements:** 

You need to run the following commands from the command line prior to launching jupyter notebooks from the same terminal so that the required libraries and paths are set:

`module use /g/data/v10/public/modules/modulefiles` 

`module load dea`  

If you find an error or bug in this notebook, please either create an 'Issue' in the Github repository, or fix it yourself and create a 'Pull' request to contribute the updated notebook back into the repository (See the repository [README](https://github.com/GeoscienceAustralia/dea-notebooks/blob/master/README.rst) for instructions on creating a Pull request).

**Date:** January 2019

**Author:** Robbi Bishop-Taylor

In [ ]:
import os
import sys
import datacube
import numpy as np
import pandas as pd
import xarray as xr
from datacube.utils import geometry
from datacube.utils.geometry import CRS
from datacube import Datacube
import matplotlib.pyplot as plt
import matplotlib

# Import external functions from dea-notebooks using relative link to Scripts
sys.path.append('../10_Scripts')
import DEAPlotting
import DEADataHandling

# Connect to datacube database
dc = datacube.Datacube(app='NDCI')

%load_ext autoreload
%autoreload 2


## Set up study areas

In [ ]:
# # Create dict with centre coordinates and buffer distances to create AOI
site_dict = {#'pamamaroo': dict(lat = -32.2738, lon = 142.4589, buffer = 5700),
             #'cawndilla': dict(lat = -32.4733, lon = 142.2291, buffer = 5700),
             'menindee': dict(lat = -32.3434, lon = 142.3279, buffer = 8700)}

# site_dict = {'cawndilla': dict(lat = -32.4733, lon = 142.2291, buffer = 5700)}
# site_dict = {'collarenebri': dict(lat = -29.5493, lon = 148.605, buffer = 1500)}
# site_dict = {'lakemurdeduke': dict(lat = -38.17725, lon = 143.897, buffer = 3500)}  
# site_dict = {'lakeinverell': dict(lat = -29.794345, lon = 151.14406, buffer = 2000)}  
# site_dict = {'pamamaroo': dict(lat = -32.2738, lon = 142.4589, buffer = 5700)}
 
# Settings for all study areas
time_range = ('2017-01-01', '2018-05-01')
resolution = (-20, 20)  
ratio = 1.5  # Output width vs height ratio for datacube spatial query
sentinel_clearprop = 0.7  # Minimum proportion of good quality imagery
sentinel_sensors = ['s2a', 's2b']  # Sentinel 2 sensors
bands = ['nbart_red_edge_1', 'nbart_red', 'nbart_green', 'nbart_nir_1', 'nbart_swir_2', 'fmask']


## Load in Sentinel 2 timeseries for each site

In [ ]:
ds_dict = {}

# Iterate through each site
for key, value in site_dict.items():  
    
    print(key)

    # Set up analysis data query using a buffer around a lat-long point (1280 x 720).
    # This simply converts a lat long to Australian Albers, then creates a square analysis region
    # by creating a square buffer around the point.
    x, y = geometry.point(value['lon'], value['lat'], CRS('WGS84')).to_crs(CRS('EPSG:3577')).points[0]
    query = {'x': (x - value['buffer'] * ratio, x + value['buffer'] * ratio),
             'y': (y - value['buffer'], y + value['buffer']),    
             'time': time_range,
             'crs': 'EPSG:3577',
             'output_crs': 'EPSG:3577',
             'resampling': 'cubic',
             'resolution': resolution} 
    
    # Load cloud free Sentinel data for all sensors (S2A, S2B) for the above query.
    sentinel_ds = DEADataHandling.load_clearsentinel2(dc=dc, query=query,
                                                      sensors=sentinel_sensors,
                                                      bands_of_interest=bands,
                                                      masked_prop=sentinel_clearprop,
                                                      mask_values=(0, 2, 3),
                                                      mask_pixel_quality=True,
                                                      mask_invalid_data=True)
    
    # Compute indices
    sentinel_ds['NDCI'] = (sentinel_ds.nbart_red_edge_1 - sentinel_ds.nbart_red) / (sentinel_ds.nbart_red_edge_1 + sentinel_ds.nbart_red)
    sentinel_ds['NDWI'] = (sentinel_ds.nbart_green - sentinel_ds.nbart_nir_1) / (sentinel_ds.nbart_green + sentinel_ds.nbart_nir_1)
    sentinel_ds['MNDWI'] = (sentinel_ds.nbart_green - sentinel_ds.nbart_swir_2) / (sentinel_ds.nbart_green + sentinel_ds.nbart_swir_2)

    # Append resulting xarray dataset to dictionary
    ds_dict[key] = sentinel_ds


In [ ]:
print(sentinel_ds.NDCI)
print(sentinel_ds.nbart_red_edge_1)
print(sentinel_ds.nbart_red)

In [ ]:
cmap = matplotlib.cm.get_cmap('jet_r')

ds.NDCI.isel(time=5).plot(cmap=cmap)

In [ ]:
ds.MNDWI.isel(time=5).plot(cmap='RdBu')

## Generate plot

In [ ]:
fig, axes = plt.subplots(len(ds_dict), 1, figsize=(8, 3 * len(ds_dict)))
cmap = matplotlib.cm.get_cmap('jet_r')

# Axes are returned outside of a list if a single subplot is requested. 
# To avoid this breaking code when we try to index into it, wrap this axes in a list first
if not isinstance(axes, list):
    axes = [axes]

for i, (key, ds) in enumerate(ds_dict.items()):     
   
    # Extract dates, total water area in km2 and summary NDCI values.
    # Take a 3-period rolling median to clean total water area counts and 
    # account for clouds/missing data on estimates of total water area
    
    sentinel_pixel_length = 20 # metres
    m_per_km = 1000 # m/km
    area_per_pixel = sentinel_pixel_length**2/m_per_km**2 #squared km area per pixel
    
    # Compute total water area based on either MNDWI or NDWI
    y = ds.MNDWI.where(ds.MNDWI > 0.0).count(dim=['x', 'y']).rolling(time=3, center=True, min_periods=1).median().values * area_per_pixel
    print(y)
    #y.isel(time=5).plot(cmap='RdBu')
    #print(ds.MNDWI.count(dim=['x', 'y']).rolling(time=3, center=True, min_periods=1).median().values())
    
#     y=ds.MNDWI.count(dim=['x', 'y']).rolling(time=3, center=True, min_periods=1).median().values * ((25 * 25) / 1e6) 
#     print(ds.MNDWI)
#     print(ds.MNDWI.count(dim=['x', 'y']).rolling(time=3, center=True, min_periods=1).median().values)
#     # y=ds.NDWI.count(dim=['x', 'y']).rolling(time=3, center=True, min_periods=1).median().values * ((25 * 25) / 1e6) 

    # Compute NDCI values for each timestep using either mean values or a percentile
    z=ds.NDCI.where(ds.MNDWI > 0.0).mean(dim=['x', 'y']).values
    # z=ds.NDCI.quantile(0.95, dim=['x', 'y']).values
    
    # Plot total water area 
    x=matplotlib.dates.date2num(ds.time.values)
    axes[i].plot_date(x=x, y=y, color='black', linestyle='-', marker='')
    axes[i].fill_between(x, 0, y, where=z >= -1, facecolor=cmap(0.7), alpha=1)
    
    # Color area under the plot based on NDCI vales: high NDCI = red
    for ii, thresh in enumerate(np.linspace(-0.1, 0.3, 100)):    
        color_vals = np.linspace(0.7, 0, 100)   
        axes[i].fill_between(x, 0, y, where=z >= thresh, facecolor=cmap(color_vals[ii]), alpha=1)

    # Set panel titles
    axes[i].set_title(key.capitalize())
    axes[i].spines['right'].set_visible(False)
    axes[i].spines['top'].set_visible(False)
    
    # If multiple panels, remove x axis from top panels
    if i < len(ds_dict) - 1:
    
        # Remove x axis 
        axes[i].axes.get_xaxis().set_ticklabels([])

# Add y axis title to middle panel
axes[-1].axes.set_ylabel('Waterbody area (km sq)')
fig.savefig('NDCI_area_plot.png', dpi=300)

In [ ]:
print(ds.NDCI.time)
cmap = matplotlib.cm.get_cmap('jet')
ds.NDCI.sel(time='2017-01-14').where(ds.MNDWI.sel(time='2017-01-14') > 0).plot(cmap=cmap)
plt.show()

ds.NDCI.sel(time='2018-02-03').where(ds.MNDWI.sel(time='2018-02-03') > 0).plot(cmap=cmap)
plt.show()

### Hacky way to create a custom colorbar for NDCI

In [ ]:
import matplotlib as mpl
import matplotlib.colors as colors

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

# The NDCI color scheme on National map uses part of the 'jet' colorscheme
cmap = truncate_colormap(plt.get_cmap('jet'), 0.3, 1.0)
norm = mpl.colors.Normalize(vmin=-0.1, vmax=0.3)

# Add a new axis to plot for colorbar
cax = fig.add_axes([0.15, 0.95, 0.7, 0.01])

# Add colorbar into new axis
cb1 = mpl.colorbar.ColorbarBase(cax, 
                                cmap=cmap,
                                norm=norm,
                                orientation='horizontal')
cb1.ax.set_title('Normalised Difference Chlorophyll Index (NDCI) for Menindee Lakes')

# Write to file
fig.savefig('NDCI_area_plot.png', dpi=300)